In [26]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests
from google.oauth2 import service_account
import os
import glob

# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'

scopes = ['https://www.googleapis.com/auth/earthengine']
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)
ee.Initialize(credentials=credentials)

# Function to fetch Sentinel-2 indices
def get_sentinel2_indices(polygon_coordinates, days_before_today):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    location_polygon = ee.Geometry.Polygon([polygon_coordinates])

    collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                  .filterDate(start_date_str, end_date_str)
                  .filterBounds(location_polygon)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                  .map(lambda image: image.select()
                       .addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))
                       .addBands(image.normalizedDifference(['B8', 'B11']).rename('NDWI'))
                       .addBands(image.expression(
                           '((NIR + 0.1) - (SWIR2 + 0.02)) / ((NIR + 0.1) + (SWIR2 + 0.02))', 
                           {'NIR': image.select('B8'), 'SWIR2': image.select('B12')}
                       ).rename('GVMI'))))

    collection_data = collection.map(lambda image: ee.Feature(None, image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=location_polygon, scale=10, maxPixels=1e9)
        .set('date', image.date().format('YYYY-MM-dd'))))

    indices_data = ee.FeatureCollection(collection_data).getInfo()['features']
    
    # Safely create the dataframe only if expected columns are present
    df = pd.DataFrame([
        {**feature['properties'], 'date': pd.to_datetime(feature['properties']['date'])} 
        for feature in indices_data
    ])
    
    # Check if the expected columns are available; if not, skip this dataset
    required_columns = ['NDVI', 'NDWI', 'GVMI']
    if not all(col in df.columns for col in required_columns):
        print("Warning: Missing one or more Sentinel-2 indices (NDVI, NDWI, GVMI). Skipping this data.")
        return pd.DataFrame(columns=['date', 'NDVI', 'NDWI', 'GVMI'])  # Return an empty dataframe
    
    # Otherwise, process the data
    df = df[['date', 'NDVI', 'NDWI', 'GVMI']]
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    df.interpolate(method='time', inplace=True)
    df.dropna(inplace=True)
    
    return df.reset_index()

# Aggregate data function as before, no changes needed here for handling the missing columns

# Function to fetch weather data including forecasts
def get_weather_data(polygon_coordinates, days_before_today, forecast_days=3):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')
    
    lon = np.mean([point[0] for point in polygon_coordinates])
    lat = np.mean([point[1] for point in polygon_coordinates])
    parameters = "T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS"
    
    nasa_power_url_hist = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameters}"
        f"&community=AG&longitude={lon}&latitude={lat}"
        f"&start={start_date_str}&end={end_date_str}&format=JSON"
    )
    response_hist = requests.get(nasa_power_url_hist)
    weather_data_hist = response_hist.json()["properties"]["parameter"]
    
    dates_hist = list(weather_data_hist["T2M"].keys())
    weather_df_hist = pd.DataFrame({
        'date': pd.to_datetime(dates_hist, format='%Y%m%d'),
        'temperature': list(weather_data_hist["T2M"].values()),
        'precipitation': list(weather_data_hist["PRECTOTCORR"].values()),
        'irradiation': list(weather_data_hist["ALLSKY_SFC_SW_DWN"].values()),
        'humidity': list(weather_data_hist["RH2M"].values()),
        'wind_speed': list(weather_data_hist["WS10M"].values()),
        'pressure': list(weather_data_hist["PS"].values())
    })
    weather_df_hist.replace(-999.00, np.nan, inplace=True)
    weather_df_hist.set_index('date', inplace=True)
    weather_df_hist.sort_index(inplace=True)
    weather_df_hist.interpolate(method='time', inplace=True)
    weather_df_hist.dropna(inplace=True)
    weather_df_hist.reset_index(inplace=True)
    
    # Create shifted features for forecast days
    for i in range(1, forecast_days + 1):
        weather_df_hist[f'temperature_t+{i}'] = weather_df_hist['temperature'].shift(-i)
        weather_df_hist[f'precipitation_t+{i}'] = weather_df_hist['precipitation'].shift(-i)
        weather_df_hist[f'irradiation_t+{i}'] = weather_df_hist['irradiation'].shift(-i)
        weather_df_hist[f'humidity_t+{i}'] = weather_df_hist['humidity'].shift(-i)
        weather_df_hist[f'wind_speed_t+{i}'] = weather_df_hist['wind_speed'].shift(-i)
        weather_df_hist[f'pressure_t+{i}'] = weather_df_hist['pressure'].shift(-i)
    weather_df_hist = weather_df_hist.iloc[:-forecast_days]
    return weather_df_hist

# Function to read and process soil moisture data files

def process_soil_moisture_file(file_path):
    # Open the file to read the lines and identify where the data starts
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Find the first line where data starts (date pattern, e.g., 'YYYY/MM/DD HH:MM')
    data_start_line = next((idx for idx, line in enumerate(lines) if line.strip().startswith('201')), None)
    
    # Check if we found the data section in the file
    if data_start_line is None:
        print(f"No data section found in {file_path}")
        return pd.DataFrame(columns=['date', 'SoilMoisture'])
    
    # Load the data from the identified line onwards
    df = pd.read_csv(
        file_path,
        skiprows=data_start_line,        # Skip lines until data starts
        delim_whitespace=True,           # Columns are whitespace-separated
        header=None,                     # No header row in the data section
        usecols=[0, 2],                  # Use only date (column 0) and soil moisture (column 2)
        names=['datetime', 'SoilMoisture']  # Name the columns for easier handling
    )

    # Convert 'datetime' to datetime, with automatic inference for various formats
    df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')
    if df['datetime'].isnull().all():
        print(f"Failed to parse dates in {file_path}")
        return pd.DataFrame(columns=['date', 'SoilMoisture'])
    
    # Convert 'SoilMoisture' to numeric, handling non-numeric values as NaN
    df['SoilMoisture'] = pd.to_numeric(df['SoilMoisture'], errors='coerce')
    if df['SoilMoisture'].isnull().all():
        print(f"No valid SoilMoisture data in {file_path}")
        return pd.DataFrame(columns=['date', 'SoilMoisture'])

    # Remove rows with NaNs in datetime or SoilMoisture
    df.dropna(subset=['datetime', 'SoilMoisture'], inplace=True)
    
    # Set 'datetime' as index and resample to daily average
    df.set_index('datetime', inplace=True)
    df_daily = df.resample('D').mean(numeric_only=True).reset_index()
    df_daily.rename(columns={'datetime': 'date'}, inplace=True)
    
    return df_daily

# Aggregate all data function
def aggregate_all_data(data_files_info, days_before_today, forecast_days):
    all_data = []
    
    for file_info in data_files_info:
        file_path = file_info['file_path']
        coordinates = file_info['coordinates']
        
        # Process soil moisture data for the location
        soil_moisture_df = process_soil_moisture_file(file_path)
        if soil_moisture_df.empty:
            print(f"No data found in soil moisture file: {file_path}")
            continue
        
        # Fetch Sentinel-2 indices for the location
        sentinel2_indices_df = get_sentinel2_indices(coordinates, days_before_today)
        if sentinel2_indices_df.empty:
            print(f"No Sentinel-2 data for coordinates: {coordinates}")
            continue
        
        # Fetch weather data for the location
        weather_df = get_weather_data(coordinates, days_before_today, forecast_days)
        if weather_df.empty:
            print(f"No weather data for coordinates: {coordinates}")
            continue
        
        # Merge data based on date for the current location
        location_data = (sentinel2_indices_df
                         .merge(soil_moisture_df, on='date', how='inner')
                         .merge(weather_df, on='date', how='inner')
                         .dropna())
        
        if location_data.empty:
            print(f"No overlapping dates after merging for file: {file_path}")
            continue
        
        # Append the location-specific merged data to the list
        all_data.append(location_data)
    
    # Concatenate data from all locations without date alignment
    if not all_data:
        raise ValueError("No data available to concatenate. Check individual files for missing or incompatible data.")
    
    full_data = pd.concat(all_data, ignore_index=True)
    return full_data

# Prepare the list of data files and their corresponding coordinates
# You need to provide this list with the actual paths and coordinates
# Example:
data_files_info = [
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.79512140110056, 4.089885950088502],
            [43.79512140110056, 4.090958833694459],
            [43.79634498472339, 4.089885950088502],
            [43.79634498472339, 4.090958833694459],
        ], # Villevielle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.63965465032482, -0.4351079463958741],
            [43.63965465032482, -0.43407797813415533],
            [43.640299091949935, -0.4351079463958741],
            [43.640299091949935, -0.43407797813415533],
        ], # Urgons
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.44097578035533, 1.8790782335418046],
            [43.44097578035533, 1.8805480840819655],
            [43.44181709737973, 1.8790782335418046],
            [43.44181709737973, 1.8805480840819655],
        ], # SaintFelixdeLauragais
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [44.146354696029114, -0.8455395698547364],
            [44.146354696029114, -0.8392953872680665],
            [44.14894137440155, -0.8455395698547364],
            [44.14894137440155, -0.8392953872680665],
        ], # Sabres
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.43737107209209, 3.400150537490845],
            [43.43737107209209, 3.4012663364410405],
            [43.43832929486419, 3.400150537490845],
            [43.43832929486419, 3.4012663364410405],
        ], # Prades-le-Lez
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.437690481368854, 3.402854204177857],
            [43.437690481368854, 3.4043991565704346],
            [43.438469521314104, 3.402854204177857],
            [43.438469521314104, 3.4043991565704346],
        ], # Pezenas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/PeyrusseGrande/SMOSMANIA_SMOSMANIA_PeyrusseGrande_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.66550453432381, 0.22144317626953128],
            [43.66550453432381, 0.2232778072357178],
            [43.66650570236179, 0.22144317626953128],
            [43.66650570236179, 0.2232778072357178],
        ], # PeyrusseGrande
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\PeyrusseGrande\SMOSMANIA_SMOSMANIA_PeyrusseGrande_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Narbonne/SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
        
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Montaut/SMOSMANIA_SMOSMANIA_Montaut_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.19177031373616, 1.6420269012451174],
            [43.19177031373616, 1.6440653800964358],
            [43.19327212688578, 1.6420269012451174],
            [43.19327212688578, 1.6440653800964358],
        ], # Montaut
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Montaut\SMOSMANIA_SMOSMANIA_Montaut_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.54607524060277, 0.8840560913085939],
            [43.54607524060277, 0.887972116470337],
            [43.54718726716286, 0.8840560913085939],
            [43.54718726716286, 0.887972116470337],
        ], # Lahas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.99368164579065, -0.04686355590820313],
            [43.99368164579065, -0.04508256912231446],
            [43.994453490832825, -0.04686355590820313],
            [43.994453490832825, -0.04508256912231446],
        ], # CreondArmagnac
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Condom/SMOSMANIA_SMOSMANIA_Condom_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.97543735840489, 0.33294796943664556],
            [43.97543735840489, 0.33539414405822754],
            [43.97763004588706, 0.33294796943664556],
            [43.97763004588706, 0.33539414405822754],
        ], # Condom
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Condom\SMOSMANIA_SMOSMANIA_Condom_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.88369667821527, 5.163574218750001],
            [43.88369667821527, 5.164990425109864],
            [43.884176109346136, 5.163574218750001],
            [43.884176109346136, 5.164990425109864],
        ],
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_static_variables.csv' 
    }, # CabrieresdAvignon

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.150000_0.250000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.350000_0.450000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.97225126706761, 15.815076828002931],
            [46.97225126706761, 15.81612825393677],
            [46.9733127794766, 15.815076828002931],
            [46.9733127794766, 15.81612825393677],
        ], # WEGENERNET 27
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.93260883554282, 15.906872749328615],
            [46.93260883554282, 15.907377004623415],
            [46.9329092118794, 15.906872749328615],
            [46.9329092118794, 15.907377004623415],
        ], # WEGENERNET 77
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.15382780227023, 0.9248793125152589],
            [42.15382780227023, 0.926499366760254],
            [42.1558082615368, 0.9248793125152589],
            [42.1558082615368, 0.926499366760254],
        ], # CamidelsNerets
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.41987666438996, 2.9409348964691167],
            [42.41987666438996, 2.942050695419312],
            [42.42105677143955, 2.9409348964691167],
            [42.42105677143955, 2.942050695419312],
        ], # Cantallops
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.155784400950616, 0.8330941200256349],
            [42.155784400950616, 0.8353686332702637],
            [42.15750234016187, 0.8330941200256349],
            [42.15750234016187, 0.8353686332702637],
        ], # ClotdelesPeresI
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_sm_0.500000_0.500000_SoilVUE10_20171101_20241110.stm',
        'coordinates': [
            [42.15523560498553, 0.8363771438598633],
            [42.15523560498553, 0.8376860618591309],
            [42.15608663437758, 0.8363771438598633],
            [42.15608663437758, 0.8376860618591309],
        ], # ClotdelesPeresII
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_sm_0.500000_0.500000_DeltaOHM-HD390.1_20171101_20241110.stm',
        'coordinates': [
            [42.19748691737416, 1.2463581562042239],
            [42.19748691737416, 1.2471413612365725],
            [42.197892278719166, 1.2463581562042239],
            [42.197892278719166, 1.2471413612365725],
        ], # ColldePaller
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [41.913718967407085, 1.5264022350311282],
            [41.913718967407085, 1.527678966522217],
            [41.91400638646574, 1.5264022350311282],
            [41.91400638646574, 1.527678966522217],
        ], # ElMiracle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.33592919742122, 3.0603575706481934],
            [42.33592919742122, 3.0639410018920903],
            [42.33748362505758, 3.0603575706481934],
            [42.33748362505758, 3.0639410018920903],
        ], # Garriguella
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.14543614475607, 0.982019240182126],
            [42.14543614475607, 0.9836822097713595],
            [42.14651799247592, 0.982019240182126],
            [42.14651799247592, 0.9836822097713595],
        ], # LosCoscolls
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.217958388113736, 1.019185781478882],
            [42.217958388113736, 1.0197973251342776],
            [42.218458965975145, 1.019185781478882],
            [42.218458965975145, 1.0197973251342776],
        ], # Pessonada
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.210655856758265, 0.8932185173034669],
            [42.210655856758265, 0.8949351310729982],
            [42.21168891029004, 0.8932185173034669],
            [42.21168891029004, 0.8949351310729982],
        ], # SerradeCostaAmpla
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_sm_0.500000_0.500000_ThetaProbe-ML2X-B_20171101_20241110.stm',
        'coordinates': [
            [44.78995234463661, -0.57828426361084],
            [44.78995234463661, -0.5760955810546876],
            [44.79204620992998, -0.57828426361084],
            [44.79204620992998, -0.5760955810546876],
        ], # FR_Aqui
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\STEMS\CAN-CT-NT\STEMS_STEMS_CAN-CT-NT_sm_0.500000_0.500000_EC5_20171101_20241110.stm',
        'coordinates': [
            [44.681729523166624, 8.625973463058473],
            [44.681729523166624, 8.627743721008303],
            [44.68297294833046, 8.625973463058473],
            [44.68297294833046, 8.627743721008303],
        ], # CAN-CT-NT
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/STEMS/CAN-CT-NT/STEMS_STEMS_CAN-CT-NT_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Gevenich\TERENO_TERENO_Gevenich_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.98609893339354, 6.32340431213379],
            [50.98609893339354, 6.330077648162843],
            [50.988908469949415, 6.32340431213379],
            [50.988908469949415, 6.330077648162843],
        ], # Gevenich
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/TERENO/Gevenich/TERENO_TERENO_Gevenich_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.92975076859899, 6.2960994243621835],
            [50.92975076859899, 6.297612190246583],
            [50.930312018942445, 6.2960994243621835],
            [50.930312018942445, 6.297612190246583],
        ], # Merzenhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.868486478093025, 6.448953151702882],
            [50.868486478093025, 6.45124912261963],
            [50.869664616471276, 6.448953151702882],
            [50.869664616471276, 6.45124912261963],
        ], # Selhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_sm_0.450000_0.450000_TRIME-EZ_20171101_20241110.stm',
        'coordinates': [
            [52.165415885421794, 14.12341833834944],
            [52.165415885421794, 14.125242240479569],
            [52.16710714333233, 14.12341833834944],
            [52.16710714333233, 14.125242240479569],
        ], # Falkenberg
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [56.48257938387106, -3.1150746345520024],
            [56.48257938387106, -3.1124567985534672],
            [56.48428556158372, -3.1150746345520024],
            [56.48428556158372, -3.1124567985534672],
        ], # Balruddery
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.86049592190234, -1.1268067359924319],
            [52.86049592190234, -1.1252081394195559],
            [52.86146755372759, -1.1268067359924319],
            [52.86146755372759, -1.1252081394195559],
        ], # BunnyPark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.105628764816664, -0.4271149635314942],
            [52.105628764816664, -0.4243040084838868],
            [52.106340450352924, -0.4271149635314942],
            [52.106340450352924, -0.4243040084838868],
        ], # Cardington
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [55.215676089352854, -1.6985893249511719],
            [55.215676089352854, -1.6943085193634035],
            [55.21752449229229, -1.6985893249511719],
            [55.21752449229229, -1.6943085193634035],
        ], # CocklePark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_static_variables.csv' 
    },
    # Add more entries for all your files
]

# Now, aggregate all data
days_before_today = 2555  # Adjust as needed
forecast_days = 3

full_data = aggregate_all_data(data_files_info, days_before_today, forecast_days)

# Display the first few rows
print(full_data.head())
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests
from google.oauth2 import service_account
import os
import glob
# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'

scopes = ['https://www.googleapis.com/auth/earthengine']
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)
ee.Initialize(credentials=credentials)

# Function to calculate daily average soil moisture with space-delimited files
def calculate_daily_average_soil_moisture(file_path):
    try:
        df = pd.read_csv(file_path, delimiter=' ', skiprows=1, names=['date', 'time', 'soil_moisture', 'col4', 'col5'], usecols=['date', 'time', 'soil_moisture'])
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y/%m/%d %H:%M', errors='coerce')
        df.drop(columns=['date', 'time'], inplace=True)
        df['date'] = df['datetime'].dt.date
        daily_avg_soil_moisture = df.groupby('date')['soil_moisture'].mean()
        return daily_avg_soil_moisture
    except pd.errors.ParserError:
        print(f"Error parsing file: {file_path}. Check the delimiter or file structure.")
        return pd.Series()

# Function to calculate average soil properties with error handling
def calculate_average_soil_properties(csv_path):
    try:
        df = pd.read_csv(csv_path, delimiter=';')
        
        # Check if required columns are present
        if not {'quantity_name', 'value'}.issubset(df.columns):
            print(f"Missing columns in {csv_path}. Expected columns: 'quantity_name' and 'value'")
            return {}

        # Filter for the desired soil properties and calculate averages
        quantities = ['saturation', 'clay fraction', 'organic carbon', 'sand fraction', 'silt fraction']
        filtered_df = df[df['quantity_name'].isin(quantities)].copy()
        filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')
        averages = filtered_df.groupby('quantity_name')['value'].mean()
        return averages.to_dict()
    except pd.errors.ParserError:
        print(f"Error parsing file: {csv_path}. Check the delimiter or file structure.")
        return {}

# Main function to combine data
def process_soil_data(data_files_info):
    combined_data = []

    for data_info in data_files_info:
        location = os.path.basename(os.path.dirname(data_info['file_path']))

        # Calculate daily average soil moisture
        daily_avg_soil_moisture = calculate_daily_average_soil_moisture(data_info['file_path'])

        # Calculate average soil properties
        soil_properties_averages = calculate_average_soil_properties(data_info['static_variables'])

        # Combine data for each date
        for date, avg_moisture in daily_avg_soil_moisture.items():
            row = {
                'Date': date,
                'Daily Average Soil Moisture': avg_moisture,
                'Location': location
            }
            row.update(soil_properties_averages)
            combined_data.append(row)

    combined_df = pd.DataFrame(combined_data)
    return combined_df


# Process data and display combined DataFrame
combined_df = process_soil_data(data_files_info)
print(combined_df)

days_before_today=365

# Convert 'Date' in combined_df to datetime to match the type in full_data
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

# Perform the merge
nn_data = pd.merge(
    full_data,
    combined_df,
    left_on=['date', 'SoilMoisture'],
    right_on=['Date', 'Daily Average Soil Moisture'],
    how='inner'
)

print(nn_data)


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_d

No Sentinel-2 data for coordinates: [[44.146354696029114, -0.8455395698547364], [44.146354696029114, -0.8392953872680665], [44.14894137440155, -0.8455395698547364], [44.14894137440155, -0.8392953872680665]]


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_d

No data section found in ./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_sm_0.500000_0.500000_CS655_20171101_20241110.stm
No data found in soil moisture file: ./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_sm_0.500000_0.500000_CS655_20171101_20241110.stm


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


...

No Sentinel-2 data for coordinates: [[52.86049592190234, -1.1268067359924319], [52.86049592190234, -1.1252081394195559], [52.86146755372759, -1.1268067359924319], [52.86146755372759, -1.1252081394195559]]


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


No Sentinel-2 data for coordinates: [[52.105628764816664, -0.4271149635314942], [52.105628764816664, -0.4243040084838868], [52.106340450352924, -0.4271149635314942], [52.106340450352924, -0.4243040084838868]]


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


No Sentinel-2 data for coordinates: [[55.215676089352854, -1.6985893249511719], [55.215676089352854, -1.6943085193634035], [55.21752449229229, -1.6985893249511719], [55.21752449229229, -1.6943085193634035]]
        date      NDVI      NDWI      GVMI  SoilMoisture  temperature  \
0 2017-11-19  0.351932 -0.022009  0.165928      0.217500        23.19   
1 2017-11-24  0.327208 -0.046502  0.166833      0.211167        26.00   
2 2017-11-29  0.313277 -0.057393  0.151082      0.203250        25.24   
3 2017-12-09  0.295641 -0.060256  0.120883      0.192542        22.69   
4 2017-12-14  0.273431 -0.083302  0.091636      0.193333        23.37   

   precipitation  irradiation  humidity  wind_speed  ...  irradiation_t+2  \
0           0.00        25.46     63.31        4.66  ...            25.21   
1           0.42        22.93     66.88        3.75  ...            21.63   
2           0.00        23.78     66.69        4.77  ...            23.94   
3           0.00        24.09     58.00       

NUOVA PROVA DATA RETRIEVAL

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests

# Define the weather data function
def get_weather_data(polygon_coordinates, days_before_today, forecast_days=3):
    try:
        lon = np.mean([point[0] for point in polygon_coordinates])
        lat = np.mean([point[1] for point in polygon_coordinates])

        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=days_before_today)

        url = (f"https://power.larc.nasa.gov/api/temporal/daily/point?"
               f"parameters=T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS&"
               f"community=AG&longitude={lon}&latitude={lat}&start={start_date:%Y%m%d}&"
               f"end={end_date:%Y%m%d}&format=JSON")
        print(f"Fetching weather data from URL: {url}")
        
        response = requests.get(url)
        response.raise_for_status()

        weather_data = response.json().get("properties", {}).get("parameter", None)
        if not weather_data:
            print("No weather data available in the response.")
            return pd.DataFrame(columns=['date', 'temperature', 'precipitation', 
                                         'irradiation', 'humidity', 'wind_speed', 'pressure'])

        df = pd.DataFrame({
            'date': pd.to_datetime(list(weather_data['T2M'].keys()), format='%Y%m%d', errors='coerce'),
            'temperature': list(weather_data['T2M'].values()),
            'precipitation': list(weather_data['PRECTOTCORR'].values()),
            'irradiation': list(weather_data['ALLSKY_SFC_SW_DWN'].values()),
            'humidity': list(weather_data['RH2M'].values()),
            'wind_speed': list(weather_data['WS10M'].values()),
            'pressure': list(weather_data['PS'].values())
        })

        df.replace(-999.00, np.nan, inplace=True)
        df.dropna(inplace=True)
        df.sort_values(by='date', inplace=True)

        for i in range(1, forecast_days + 1):
            df[f'temperature_t+{i}'] = df['temperature'].shift(-i)
            df[f'precipitation_t+{i}'] = df['precipitation'].shift(-i)
            df[f'irradiation_t+{i}'] = df['irradiation'].shift(-i)
            df[f'humidity_t+{i}'] = df['humidity'].shift(-i)
            df[f'wind_speed_t+{i}'] = df['wind_speed'].shift(-i)
            df[f'pressure_t+{i}'] = df['pressure'].shift(-i)

        df = df.iloc[:-forecast_days] if forecast_days > 0 else df
        return df.reset_index(drop=True)
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return pd.DataFrame(columns=['date', 'temperature', 'precipitation', 
                                     'irradiation', 'humidity', 'wind_speed', 'pressure'])

# Define function to process and combine data for all locations
def process_and_combine_all_locations(data_files_info, days_before_today=3000, forecast_days=3):
    processed_dfs = []
    for info in data_files_info:
        try:
            soil_moisture_df = process_soil_moisture_file(info['file_path'])
            soil_moisture_df['date'] = pd.to_datetime(soil_moisture_df['date'], errors='coerce')

            soil_properties = calculate_average_soil_properties(info['static_variables'])

            sentinel2_df = get_sentinel2_indices(info['coordinates'], days_before_today)
            if sentinel2_df.empty:
                print(f"No Sentinel-2 data for coordinates: {info['coordinates']}")
                continue
            sentinel2_df['date'] = pd.to_datetime(sentinel2_df['date'], errors='coerce')

            weather_df = get_weather_data(info['coordinates'], days_before_today, forecast_days)
            if weather_df.empty:
                print(f"No weather data for coordinates: {info['coordinates']}")
                continue
            weather_df['date'] = pd.to_datetime(weather_df['date'], errors='coerce')

            combined_df = pd.merge(soil_moisture_df, sentinel2_df, on='date', how='inner')
            combined_df = pd.merge(combined_df, weather_df, on='date', how='inner')

            for key, value in soil_properties.items():
                combined_df[key] = value

            processed_dfs.append(combined_df)
        except Exception as e:
            print(f"Error processing location {info.get('file_path', 'Unknown')}: {e}")
            continue

    return pd.concat(processed_dfs, ignore_index=True) if processed_dfs else pd.DataFrame()





data_files_info = [
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.79512140110056, 4.089885950088502],
            [43.79512140110056, 4.090958833694459],
            [43.79634498472339, 4.089885950088502],
            [43.79634498472339, 4.090958833694459],
        ], # Villevielle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.63965465032482, -0.4351079463958741],
            [43.63965465032482, -0.43407797813415533],
            [43.640299091949935, -0.4351079463958741],
            [43.640299091949935, -0.43407797813415533],
        ], # Urgons
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.44097578035533, 1.8790782335418046],
            [43.44097578035533, 1.8805480840819655],
            [43.44181709737973, 1.8790782335418046],
            [43.44181709737973, 1.8805480840819655],
        ], # SaintFelixdeLauragais
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [44.146354696029114, -0.8455395698547364],
            [44.146354696029114, -0.8392953872680665],
            [44.14894137440155, -0.8455395698547364],
            [44.14894137440155, -0.8392953872680665],
        ], # Sabres
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.43737107209209, 3.400150537490845],
            [43.43737107209209, 3.4012663364410405],
            [43.43832929486419, 3.400150537490845],
            [43.43832929486419, 3.4012663364410405],
        ], # Prades-le-Lez
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.437690481368854, 3.402854204177857],
            [43.437690481368854, 3.4043991565704346],
            [43.438469521314104, 3.402854204177857],
            [43.438469521314104, 3.4043991565704346],
        ], # Pezenas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/PeyrusseGrande/SMOSMANIA_SMOSMANIA_PeyrusseGrande_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.66550453432381, 0.22144317626953128],
            [43.66550453432381, 0.2232778072357178],
            [43.66650570236179, 0.22144317626953128],
            [43.66650570236179, 0.2232778072357178],
        ], # PeyrusseGrande
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\PeyrusseGrande\SMOSMANIA_SMOSMANIA_PeyrusseGrande_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Narbonne/SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
        
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Montaut/SMOSMANIA_SMOSMANIA_Montaut_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.19177031373616, 1.6420269012451174],
            [43.19177031373616, 1.6440653800964358],
            [43.19327212688578, 1.6420269012451174],
            [43.19327212688578, 1.6440653800964358],
        ], # Montaut
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Montaut\SMOSMANIA_SMOSMANIA_Montaut_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.54607524060277, 0.8840560913085939],
            [43.54607524060277, 0.887972116470337],
            [43.54718726716286, 0.8840560913085939],
            [43.54718726716286, 0.887972116470337],
        ], # Lahas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.99368164579065, -0.04686355590820313],
            [43.99368164579065, -0.04508256912231446],
            [43.994453490832825, -0.04686355590820313],
            [43.994453490832825, -0.04508256912231446],
        ], # CreondArmagnac
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Condom/SMOSMANIA_SMOSMANIA_Condom_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.97543735840489, 0.33294796943664556],
            [43.97543735840489, 0.33539414405822754],
            [43.97763004588706, 0.33294796943664556],
            [43.97763004588706, 0.33539414405822754],
        ], # Condom
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Condom\SMOSMANIA_SMOSMANIA_Condom_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.88369667821527, 5.163574218750001],
            [43.88369667821527, 5.164990425109864],
            [43.884176109346136, 5.163574218750001],
            [43.884176109346136, 5.164990425109864],
        ],
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_static_variables.csv' 
    }, # CabrieresdAvignon

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.150000_0.250000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.350000_0.450000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.97225126706761, 15.815076828002931],
            [46.97225126706761, 15.81612825393677],
            [46.9733127794766, 15.815076828002931],
            [46.9733127794766, 15.81612825393677],
        ], # WEGENERNET 27
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.93260883554282, 15.906872749328615],
            [46.93260883554282, 15.907377004623415],
            [46.9329092118794, 15.906872749328615],
            [46.9329092118794, 15.907377004623415],
        ], # WEGENERNET 77
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.15382780227023, 0.9248793125152589],
            [42.15382780227023, 0.926499366760254],
            [42.1558082615368, 0.9248793125152589],
            [42.1558082615368, 0.926499366760254],
        ], # CamidelsNerets
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.41987666438996, 2.9409348964691167],
            [42.41987666438996, 2.942050695419312],
            [42.42105677143955, 2.9409348964691167],
            [42.42105677143955, 2.942050695419312],
        ], # Cantallops
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.155784400950616, 0.8330941200256349],
            [42.155784400950616, 0.8353686332702637],
            [42.15750234016187, 0.8330941200256349],
            [42.15750234016187, 0.8353686332702637],
        ], # ClotdelesPeresI
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_sm_0.500000_0.500000_SoilVUE10_20171101_20241110.stm',
        'coordinates': [
            [42.15523560498553, 0.8363771438598633],
            [42.15523560498553, 0.8376860618591309],
            [42.15608663437758, 0.8363771438598633],
            [42.15608663437758, 0.8376860618591309],
        ], # ClotdelesPeresII
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_sm_0.500000_0.500000_DeltaOHM-HD390.1_20171101_20241110.stm',
        'coordinates': [
            [42.19748691737416, 1.2463581562042239],
            [42.19748691737416, 1.2471413612365725],
            [42.197892278719166, 1.2463581562042239],
            [42.197892278719166, 1.2471413612365725],
        ], # ColldePaller
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [41.913718967407085, 1.5264022350311282],
            [41.913718967407085, 1.527678966522217],
            [41.91400638646574, 1.5264022350311282],
            [41.91400638646574, 1.527678966522217],
        ], # ElMiracle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.33592919742122, 3.0603575706481934],
            [42.33592919742122, 3.0639410018920903],
            [42.33748362505758, 3.0603575706481934],
            [42.33748362505758, 3.0639410018920903],
        ], # Garriguella
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.14543614475607, 0.982019240182126],
            [42.14543614475607, 0.9836822097713595],
            [42.14651799247592, 0.982019240182126],
            [42.14651799247592, 0.9836822097713595],
        ], # LosCoscolls
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.217958388113736, 1.019185781478882],
            [42.217958388113736, 1.0197973251342776],
            [42.218458965975145, 1.019185781478882],
            [42.218458965975145, 1.0197973251342776],
        ], # Pessonada
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.210655856758265, 0.8932185173034669],
            [42.210655856758265, 0.8949351310729982],
            [42.21168891029004, 0.8932185173034669],
            [42.21168891029004, 0.8949351310729982],
        ], # SerradeCostaAmpla
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_sm_0.500000_0.500000_ThetaProbe-ML2X-B_20171101_20241110.stm',
        'coordinates': [
            [44.78995234463661, -0.57828426361084],
            [44.78995234463661, -0.5760955810546876],
            [44.79204620992998, -0.57828426361084],
            [44.79204620992998, -0.5760955810546876],
        ], # FR_Aqui
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\STEMS\CAN-CT-NT\STEMS_STEMS_CAN-CT-NT_sm_0.500000_0.500000_EC5_20171101_20241110.stm',
        'coordinates': [
            [44.681729523166624, 8.625973463058473],
            [44.681729523166624, 8.627743721008303],
            [44.68297294833046, 8.625973463058473],
            [44.68297294833046, 8.627743721008303],
        ], # CAN-CT-NT
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/STEMS/CAN-CT-NT/STEMS_STEMS_CAN-CT-NT_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Gevenich\TERENO_TERENO_Gevenich_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.98609893339354, 6.32340431213379],
            [50.98609893339354, 6.330077648162843],
            [50.988908469949415, 6.32340431213379],
            [50.988908469949415, 6.330077648162843],
        ], # Gevenich
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/TERENO/Gevenich/TERENO_TERENO_Gevenich_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.92975076859899, 6.2960994243621835],
            [50.92975076859899, 6.297612190246583],
            [50.930312018942445, 6.2960994243621835],
            [50.930312018942445, 6.297612190246583],
        ], # Merzenhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.868486478093025, 6.448953151702882],
            [50.868486478093025, 6.45124912261963],
            [50.869664616471276, 6.448953151702882],
            [50.869664616471276, 6.45124912261963],
        ], # Selhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_sm_0.450000_0.450000_TRIME-EZ_20171101_20241110.stm',
        'coordinates': [
            [52.165415885421794, 14.12341833834944],
            [52.165415885421794, 14.125242240479569],
            [52.16710714333233, 14.12341833834944],
            [52.16710714333233, 14.125242240479569],
        ], # Falkenberg
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [56.48257938387106, -3.1150746345520024],
            [56.48257938387106, -3.1124567985534672],
            [56.48428556158372, -3.1150746345520024],
            [56.48428556158372, -3.1124567985534672],
        ], # Balruddery
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.86049592190234, -1.1268067359924319],
            [52.86049592190234, -1.1252081394195559],
            [52.86146755372759, -1.1268067359924319],
            [52.86146755372759, -1.1252081394195559],
        ], # BunnyPark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.105628764816664, -0.4271149635314942],
            [52.105628764816664, -0.4243040084838868],
            [52.106340450352924, -0.4271149635314942],
            [52.106340450352924, -0.4243040084838868],
        ], # Cardington
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [55.215676089352854, -1.6985893249511719],
            [55.215676089352854, -1.6943085193634035],
            [55.21752449229229, -1.6985893249511719],
            [55.21752449229229, -1.6943085193634035],
        ], # CocklePark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_static_variables.csv' 
    },
    # Add more entries for all your files
]

final_combined_df = process_and_combine_all_locations(data_files_info)


# Inspect the final concatenated DataFrame
print(final_combined_df)

C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


Fetching weather data from URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS&community=AG&longitude=43.795733192911975&latitude=4.09042239189148&start=20160829&end=20241115&format=JSON


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


Fetching weather data from URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS&community=AG&longitude=43.63997687113738&latitude=-0.4345929622650147&start=20160829&end=20241115&format=JSON


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


...

C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


No Sentinel-2 data available for the given parameters: [[52.105628764816664, -0.4271149635314942], [52.105628764816664, -0.4243040084838868], [52.106340450352924, -0.4271149635314942], [52.106340450352924, -0.4243040084838868]]
No Sentinel-2 data for coordinates: [[52.105628764816664, -0.4271149635314942], [52.105628764816664, -0.4243040084838868], [52.106340450352924, -0.4271149635314942], [52.106340450352924, -0.4243040084838868]]


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:138: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1488461155.py:148: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True, errors='coerce')


No Sentinel-2 data available for the given parameters: [[55.215676089352854, -1.6985893249511719], [55.215676089352854, -1.6943085193634035], [55.21752449229229, -1.6985893249511719], [55.21752449229229, -1.6943085193634035]]
No Sentinel-2 data for coordinates: [[55.215676089352854, -1.6985893249511719], [55.215676089352854, -1.6943085193634035], [55.21752449229229, -1.6985893249511719], [55.21752449229229, -1.6943085193634035]]
            date  SoilMoisture      NDVI      NDWI      GVMI  temperature  \
0     2017-11-04      0.088292  0.138985 -0.040368  0.114831        25.18   
1     2017-11-09      0.247458  0.271704  0.088411  0.351790        24.90   
2     2017-11-14      0.230333  0.371808 -0.017093  0.194411        24.44   
3     2017-11-19      0.217500  0.351932 -0.022009  0.165928        23.19   
4     2017-11-24      0.211167  0.327208 -0.046502  0.166833        26.00   
...          ...           ...       ...       ...       ...          ...   
10038 2022-02-23      0.3540

C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\3065885146.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(processed_dfs, ignore_index=True) if processed_dfs else pd.DataFrame()


In [27]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta

# Assuming nn_data is already loaded as a DataFrame with a 'date' and 'SoilMoisture' columns

# Step 1: Data Preparation
# Drop the 'date' column and separate features and target (SoilMoisture)
nn_data = nn_data.select_dtypes(include=[np.number])  # Select only numeric columns to avoid any Timestamp issues
X = nn_data.drop(columns=['SoilMoisture', 'Location'])
y = nn_data['SoilMoisture']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 2: Define the Neural Network Model
class SoilMoistureNN(nn.Module):
    def __init__(self, input_size):
        super(SoilMoistureNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        return self.output(x)

# Step 3: Initialize Model, Loss, and Optimizer
input_size = X_train.shape[1]
model = SoilMoistureNN(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Step 4: Train the Model
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Step 5: Evaluate the Model
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f"Test Loss: {test_loss.item()}")

# Step 6: Generate Irrigation Suggestions
# Predict soil moisture for the next three days and generate suggestions
forecast_days = 3
X_future = X_test_tensor[-forecast_days:]  # Using the last data points as the basis for forecasting

with torch.no_grad():
    future_predictions = model(X_future).numpy().flatten()

# Define the field capacity and threshold for irrigation
default_field_capacity = 0.3
FTSW_thresh = 0.7
VWC_thresh = FTSW_thresh * default_field_capacity

# Create future dates
future_dates = [datetime.today() + timedelta(days=i) for i in range(1, forecast_days + 1)]

# Build the output DataFrame
results = pd.DataFrame({
    'Date': future_dates,
    'Predicted_SoilMoisture': future_predictions
})

# Add irrigation suggestions based on the threshold
results['Irrigate'] = results['Predicted_SoilMoisture'] < VWC_thresh
results['Suggest'] = results['Irrigate'].apply(lambda x: 'Irrigate' if x else 'No Action')

# Display the results
print(results[['Date', 'Predicted_SoilMoisture', 'Suggest']])


KeyError: "['Location'] not found in axis"

In [ ]:
import joblib
import torch
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define the model class
class SoilMoistureNN(nn.Module):
    def __init__(self, input_size):
        super(SoilMoistureNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        return self.output(x)

# Function to fetch data for prediction
def get_data_for_prediction(polygon_coordinates, days_before_today, forecast_days):
    sentinel2_indices_df = get_sentinel2_indices(polygon_coordinates, days_before_today)
    weather_df = get_weather_data(polygon_coordinates, days_before_today, forecast_days)
    sentinel2_indices_df['date'] = pd.to_datetime(sentinel2_indices_df['date'])
    weather_df['date'] = pd.to_datetime(weather_df['date'])
    
    # Merge and clean the data
    data = (sentinel2_indices_df
            .merge(weather_df, on='date', how='inner')
            .dropna())
    return data

# Prediction function
def predict_soil_moisture(polygon_coordinates, default_field_capacity=0.3):
    days_before_today = 30
    forecast_days = 3

    # Retrieve data for prediction
    data = get_data_for_prediction(polygon_coordinates, days_before_today, forecast_days)
    
    # Select the last few rows as the base for future prediction
    X_new = data.drop(columns=['date']).values[-forecast_days:]
    
    # Load scaler and model
    scaler = joblib.load('scaler.pkl')
    model = SoilMoistureNN(X_new.shape[1])
    model.load_state_dict(torch.load('soil_moisture_model.pth'))
    model.eval()
    
    # Scale the data
    X_new_scaled = scaler.transform(X_new)
    X_new_tensor = torch.tensor(X_new_scaled, dtype=torch.float32)

    # Make predictions
    with torch.no_grad():
        y_pred = model(X_new_tensor).numpy().flatten()

    # Adjust forecast days if necessary
    if len(y_pred) != forecast_days:
        forecast_days = len(y_pred)
    
    # Create future dates for prediction output
    future_dates = [datetime.today() + timedelta(days=i) for i in range(1, forecast_days + 1)]

    # Construct the output DataFrame
    results = pd.DataFrame({
        'Date': future_dates,
        'Predicted_SoilMoisture': y_pred
    })

    # Use the default field capacity for irrigation threshold calculation
    field_capacity = default_field_capacity
    FTSW_thresh = 0.7
    VWC_thresh = FTSW_thresh * field_capacity

    # Add irrigation suggestion based on the threshold
    results['Irrigate'] = results['Predicted_SoilMoisture'] < VWC_thresh

    # Fetch precipitation forecast for the next three days from data
    forecast_precipitation = data[['date', 'precipitation_t+1', 'precipitation_t+2', 'precipitation_t+3']].tail(forecast_days)
    forecast_precipitation['Total_Precipitation_Next_3_Days'] = forecast_precipitation[['precipitation_t+1', 'precipitation_t+2', 'precipitation_t+3']].sum(axis=1)
    
    # Merge forecast precipitation with results
    results = results.merge(forecast_precipitation[['date', 'Total_Precipitation_Next_3_Days']], left_on='Date', right_on='date', how='left')

    # Add a final suggestion based on precipitation
    results['Suggest'] = results.apply(lambda row: 'Wait' if row['Total_Precipitation_Next_3_Days'] > 5 else ('Irrigate' if row['Irrigate'] else 'No Action'), axis=1)

    # Return the relevant columns
    return results[['Date', 'Predicted_SoilMoisture', 'Suggest']]

# Example usage with new coordinates
new_coordinates = [
    [12.021280, 45.251061],
    [12.021719, 45.250949],
    [12.022229, 45.251746],
    [12.022891, 45.251549],
    [12.023380, 45.252315],
    [12.023846, 45.252183],
    [12.024085, 45.252617],
    [12.022100, 45.252718],
    [12.021280, 45.251061]  # Closing the polygon by repeating the first point
]

# Run prediction
results = predict_soil_moisture(new_coordinates)
print(results)


C:\Users\serafino\AppData\Local\Temp\ipykernel_1628\1567355579.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('soil_moisture_model.pth

RuntimeError: Error(s) in loading state_dict for SoilMoistureNN:
	size mismatch for fc1.weight: copying a param with shape torch.Size([128, 27]) from checkpoint, the shape in current model is torch.Size([128, 36]).

In [ ]:
#########THIS WORKS######
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests
from google.oauth2 import service_account
import os
import glob
# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'

scopes = ['https://www.googleapis.com/auth/earthengine']
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)
ee.Initialize(credentials=credentials)

# Function to calculate daily average soil moisture with space-delimited files
def calculate_daily_average_soil_moisture(file_path):
    try:
        df = pd.read_csv(file_path, delimiter=' ', skiprows=1, names=['date', 'time', 'soil_moisture', 'col4', 'col5'], usecols=['date', 'time', 'soil_moisture'])
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y/%m/%d %H:%M', errors='coerce')
        df.drop(columns=['date', 'time'], inplace=True)
        df['date'] = df['datetime'].dt.date
        daily_avg_soil_moisture = df.groupby('date')['soil_moisture'].mean()
        return daily_avg_soil_moisture
    except pd.errors.ParserError:
        print(f"Error parsing file: {file_path}. Check the delimiter or file structure.")
        return pd.Series()

# Function to calculate average soil properties with error handling
def calculate_average_soil_properties(csv_path):
    try:
        df = pd.read_csv(csv_path, delimiter=';')
        
        # Check if required columns are present
        if not {'quantity_name', 'value'}.issubset(df.columns):
            print(f"Missing columns in {csv_path}. Expected columns: 'quantity_name' and 'value'")
            return {}

        # Filter for the desired soil properties and calculate averages
        quantities = ['saturation', 'clay fraction', 'organic carbon', 'sand fraction', 'silt fraction']
        filtered_df = df[df['quantity_name'].isin(quantities)].copy()
        filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')
        averages = filtered_df.groupby('quantity_name')['value'].mean()
        return averages.to_dict()
    except pd.errors.ParserError:
        print(f"Error parsing file: {csv_path}. Check the delimiter or file structure.")
        return {}

# Main function to combine data
def process_soil_data(data_files_info):
    combined_data = []

    for data_info in data_files_info:
        location = os.path.basename(os.path.dirname(data_info['file_path']))

        # Calculate daily average soil moisture
        daily_avg_soil_moisture = calculate_daily_average_soil_moisture(data_info['file_path'])

        # Calculate average soil properties
        soil_properties_averages = calculate_average_soil_properties(data_info['static_variables'])

        # Combine data for each date
        for date, avg_moisture in daily_avg_soil_moisture.items():
            row = {
                'Date': date,
                'Daily Average Soil Moisture': avg_moisture,
                'Location': location
            }
            row.update(soil_properties_averages)
            combined_data.append(row)

    combined_df = pd.DataFrame(combined_data)
    return combined_df


# Process data and display combined DataFrame
combined_df = process_soil_data(data_files_info)
print(combined_df)

days_before_today=365


Missing columns in ./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_static_variables.csv. Expected columns: 'quantity_name' and 'value'
             Date  Daily Average Soil Moisture     Location  clay fraction  \
0      2017-11-01                     0.087500  Villevielle          13.48   
1      2017-11-02                     0.088000  Villevielle          13.48   
2      2017-11-03                     0.087750  Villevielle          13.48   
3      2017-11-04                     0.088292  Villevielle          13.48   
4      2017-11-05                     0.237833  Villevielle          13.48   
...           ...                          ...          ...            ...   
63305  2022-12-27                     0.521000   CocklePark          25.00   
63306  2022-12-28                     0.531000   CocklePark          25.00   
63307  2022-12-29                     0.493000   CocklePark          25.00   
63308  2022-12-30     

In [ ]:
import ee
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests
from google.oauth2 import service_account
import os
import glob
# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'

scopes = ['https://www.googleapis.com/auth/earthengine']
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)
ee.Initialize(credentials=credentials)

# Function to calculate daily average soil moisture with space-delimited files
def calculate_daily_average_soil_moisture(file_path):
    try:
        df = pd.read_csv(file_path, delimiter=' ', skiprows=1, names=['date', 'time', 'soil_moisture', 'col4', 'col5'], usecols=['date', 'time', 'soil_moisture'])
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y/%m/%d %H:%M', errors='coerce')
        df.drop(columns=['date', 'time'], inplace=True)
        df['date'] = df['datetime'].dt.date
        daily_avg_soil_moisture = df.groupby('date')['soil_moisture'].mean()
        return daily_avg_soil_moisture
    except pd.errors.ParserError:
        print(f"Error parsing file: {file_path}. Check the delimiter or file structure.")
        return pd.Series()

# Function to calculate average soil properties with error handling
def calculate_average_soil_properties(csv_path):
    try:
        df = pd.read_csv(csv_path, delimiter=';')
        
        # Check if required columns are present
        if not {'quantity_name', 'value'}.issubset(df.columns):
            print(f"Missing columns in {csv_path}. Expected columns: 'quantity_name' and 'value'")
            return {}

        # Filter for the desired soil properties and calculate averages
        quantities = ['saturation', 'clay fraction', 'organic carbon', 'sand fraction', 'silt fraction']
        filtered_df = df[df['quantity_name'].isin(quantities)].copy()
        filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')
        averages = filtered_df.groupby('quantity_name')['value'].mean()
        return averages.to_dict()
    except pd.errors.ParserError:
        print(f"Error parsing file: {csv_path}. Check the delimiter or file structure.")
        return {}

# Main function to combine data
def process_soil_data(data_files_info):
    combined_data = []

    for data_info in data_files_info:
        location = os.path.basename(os.path.dirname(data_info['file_path']))

        # Calculate daily average soil moisture
        daily_avg_soil_moisture = calculate_daily_average_soil_moisture(data_info['file_path'])

        # Calculate average soil properties
        soil_properties_averages = calculate_average_soil_properties(data_info['static_variables'])

        # Combine data for each date
        for date, avg_moisture in daily_avg_soil_moisture.items():
            row = {
                'Date': date,
                'Daily Average Soil Moisture': avg_moisture,
                'Location': location
            }
            row.update(soil_properties_averages)
            combined_data.append(row)

    combined_df = pd.DataFrame(combined_data)
    return combined_df


# Process data and display combined DataFrame
combined_df = process_soil_data(data_files_info)
print(combined_df)

days_before_today=365
# Function to process soil data using the new methods
def process_soil_data(data_files_info):
    combined_data = []

    for data_info in data_files_info:
        location = os.path.basename(os.path.dirname(data_info['file_path']))

        # Calculate daily average soil moisture
        daily_avg_soil_moisture = calculate_daily_average_soil_moisture(data_info['file_path'])
        if daily_avg_soil_moisture.empty:
            print(f"No valid soil moisture data for {location}. Skipping.")
            continue

        # Calculate average soil properties
        soil_properties_averages = calculate_average_soil_properties(data_info['static_variables'])
        
        # Combine data for each date
        for date, avg_moisture in daily_avg_soil_moisture.items():
            row = {
                'date': pd.to_datetime(date),  # Ensure date is in datetime format
                'Daily Average Soil Moisture': avg_moisture,
                'Location': location
            }
            row.update(soil_properties_averages)
            combined_data.append(row)

    combined_df = pd.DataFrame(combined_data)
    return combined_df
# Modify aggregate_all_data to store each location's data sequentially
def aggregate_all_data(data_files_info, days_before_today, forecast_days):
    all_data = []

    for file_info in data_files_info:
        coordinates = file_info['coordinates']
        location = os.path.basename(os.path.dirname(file_info['file_path']))

        # Process soil data for the location
        soil_data_df = process_soil_data([file_info])
        if soil_data_df.empty:
            print(f"No valid soil data for {location}. Skipping.")
            continue
        
        # Fetch Sentinel-2 indices for the location
        sentinel2_indices_df = get_sentinel2_indices(coordinates, days_before_today)
        if sentinel2_indices_df.empty:
            print(f"No Sentinel-2 data for {location}. Skipping.")
            continue

        # Fetch weather data for the location
        weather_df = get_weather_data(coordinates, days_before_today, forecast_days)
        if weather_df.empty:
            print(f"No weather data for {location}. Skipping.")
            continue

        # Merge soil, Sentinel-2, and weather data based on 'date' for each location
        location_data = (sentinel2_indices_df
                         .merge(soil_data_df, on='date', how='inner')
                         .merge(weather_df, on='date', how='inner')
                         .dropna())
        
        # Add a column for location to distinguish data when concatenated
        location_data['Location'] = location

        print(f"Head of Merged Data for location {location}:\n", location_data.head(), "\n")

        # Append the location-specific data to the list
        all_data.append(location_data)
    
    # Concatenate all locations' data one after the other, without alignment
    if not all_data:
        raise ValueError("No data available to concatenate. Check individual files for missing or incompatible data.")
    
    full_data = pd.concat(all_data, ignore_index=True)
    return full_data
data_files_info = [
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.79512140110056, 4.089885950088502],
            [43.79512140110056, 4.090958833694459],
            [43.79634498472339, 4.089885950088502],
            [43.79634498472339, 4.090958833694459],
        ], # Villevielle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Villevielle\SMOSMANIA_SMOSMANIA_Villevielle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.63965465032482, -0.4351079463958741],
            [43.63965465032482, -0.43407797813415533],
            [43.640299091949935, -0.4351079463958741],
            [43.640299091949935, -0.43407797813415533],
        ], # Urgons
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Urgons\SMOSMANIA_SMOSMANIA_Urgons_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.82578087836081, 1.1749470233917239],
            [43.82578087836081, 1.1771786212921145],
            [43.82712767623998, 1.1749470233917239],
            [43.82712767623998, 1.1771786212921145],
        ], # Savenes 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Savenes\SMOSMANIA_SMOSMANIA_Savenes_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.44097578035533, 1.8790782335418046],
            [43.44097578035533, 1.8805480840819655],
            [43.44181709737973, 1.8790782335418046],
            [43.44181709737973, 1.8805480840819655],
        ], # SaintFelixdeLauragais
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\SaintFelixdeLauragais\SMOSMANIA_SMOSMANIA_SaintFelixdeLauragais_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [44.146354696029114, -0.8455395698547364],
            [44.146354696029114, -0.8392953872680665],
            [44.14894137440155, -0.8455395698547364],
            [44.14894137440155, -0.8392953872680665],
        ], # Sabres
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Sabres\SMOSMANIA_SMOSMANIA_Sabres_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.43737107209209, 3.400150537490845],
            [43.43737107209209, 3.4012663364410405],
            [43.43832929486419, 3.400150537490845],
            [43.43832929486419, 3.4012663364410405],
        ], # Prades-le-Lez
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Prades-le-Lez\SMOSMANIA_SMOSMANIA_Prades-le-Lez_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.437690481368854, 3.402854204177857],
            [43.437690481368854, 3.4043991565704346],
            [43.438469521314104, 3.402854204177857],
            [43.438469521314104, 3.4043991565704346],
        ], # Pezenas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Pezenas\SMOSMANIA_SMOSMANIA_Pezenas_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/PeyrusseGrande/SMOSMANIA_SMOSMANIA_PeyrusseGrande_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.66550453432381, 0.22144317626953128],
            [43.66550453432381, 0.2232778072357178],
            [43.66650570236179, 0.22144317626953128],
            [43.66650570236179, 0.2232778072357178],
        ], # PeyrusseGrande
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\PeyrusseGrande\SMOSMANIA_SMOSMANIA_PeyrusseGrande_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Narbonne/SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 1
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
        
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.14930534152856, 2.9562449455261235],
            [43.14930534152856, 2.9568994045257573],
            [43.150150703530116, 2.9562449455261235],
            [43.150150703530116, 2.9568994045257573],
        ], # Narbonne 2
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Narbonne\SMOSMANIA_SMOSMANIA_Narbonne_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Montaut/SMOSMANIA_SMOSMANIA_Montaut_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.19177031373616, 1.6420269012451174],
            [43.19177031373616, 1.6440653800964358],
            [43.19327212688578, 1.6420269012451174],
            [43.19327212688578, 1.6440653800964358],
        ], # Montaut
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Montaut\SMOSMANIA_SMOSMANIA_Montaut_static_variables.csv' 
    },
    
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.54607524060277, 0.8840560913085939],
            [43.54607524060277, 0.887972116470337],
            [43.54718726716286, 0.8840560913085939],
            [43.54718726716286, 0.887972116470337],
        ], # Lahas
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Lahas\SMOSMANIA_SMOSMANIA_Lahas_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_sm_0.300000_0.300000_ThetaProbe-ML3_20171101_20241110.stm',
        'coordinates': [
            [43.99368164579065, -0.04686355590820313],
            [43.99368164579065, -0.04508256912231446],
            [43.994453490832825, -0.04686355590820313],
            [43.994453490832825, -0.04508256912231446],
        ], # CreondArmagnac
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CreondArmagnac\SMOSMANIA_SMOSMANIA_CreondArmagnac_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/SMOSMANIA/Condom/SMOSMANIA_SMOSMANIA_Condom_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.97543735840489, 0.33294796943664556],
            [43.97543735840489, 0.33539414405822754],
            [43.97763004588706, 0.33294796943664556],
            [43.97763004588706, 0.33539414405822754],
        ], # Condom
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\Condom\SMOSMANIA_SMOSMANIA_Condom_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_sm_0.300000_0.300000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.88369667821527, 5.163574218750001],
            [43.88369667821527, 5.164990425109864],
            [43.884176109346136, 5.163574218750001],
            [43.884176109346136, 5.164990425109864],
        ],
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\SMOSMANIA\CabrieresdAvignon\SMOSMANIA_SMOSMANIA_CabrieresdAvignon_static_variables.csv' 
    }, # CabrieresdAvignon

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.150000_0.250000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_sm_0.350000_0.450000_ThetaProbe-ML2X_20171101_20241110.stm',
        'coordinates': [
            [43.31968276747214, 12.699025869369509],
            [43.31968276747214, 12.700774669647219],
            [43.32132192691461, 12.699025869369509],
            [43.32132192691461, 12.700774669647219],
        ], # Torre-Olmo
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\UMBRIA\Torre-Olmo\UMBRIA_UMBRIA_Torre-Olmo_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.97225126706761, 15.815076828002931],
            [46.97225126706761, 15.81612825393677],
            [46.9733127794766, 15.815076828002931],
            [46.9733127794766, 15.81612825393677],
        ], # WEGENERNET 27
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\27\WEGENERNET_WEGENERNET_27_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_sm_0.300000_0.300000_pF-Meter_20171101_20241110.stm',
        'coordinates': [
            [46.93260883554282, 15.906872749328615],
            [46.93260883554282, 15.907377004623415],
            [46.9329092118794, 15.906872749328615],
            [46.9329092118794, 15.907377004623415],
        ], # WEGENERNET 77
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\WEGENERNET\77\WEGENERNET_WEGENERNET_77_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.15382780227023, 0.9248793125152589],
            [42.15382780227023, 0.926499366760254],
            [42.1558082615368, 0.9248793125152589],
            [42.1558082615368, 0.926499366760254],
        ], # CamidelsNerets
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\CamidelsNerets\XMS-CAT_XMS-CAT_CamidelsNerets_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.41987666438996, 2.9409348964691167],
            [42.41987666438996, 2.942050695419312],
            [42.42105677143955, 2.9409348964691167],
            [42.42105677143955, 2.942050695419312],
        ], # Cantallops
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Cantallops\XMS-CAT_XMS-CAT_Cantallops_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.155784400950616, 0.8330941200256349],
            [42.155784400950616, 0.8353686332702637],
            [42.15750234016187, 0.8330941200256349],
            [42.15750234016187, 0.8353686332702637],
        ], # ClotdelesPeresI
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresI\XMS-CAT_XMS-CAT_ClotdelesPeresI_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_sm_0.500000_0.500000_SoilVUE10_20171101_20241110.stm',
        'coordinates': [
            [42.15523560498553, 0.8363771438598633],
            [42.15523560498553, 0.8376860618591309],
            [42.15608663437758, 0.8363771438598633],
            [42.15608663437758, 0.8376860618591309],
        ], # ClotdelesPeresII
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ClotdelesPeresII\XMS-CAT_XMS-CAT_ClotdelesPeresII_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_sm_0.500000_0.500000_DeltaOHM-HD390.1_20171101_20241110.stm',
        'coordinates': [
            [42.19748691737416, 1.2463581562042239],
            [42.19748691737416, 1.2471413612365725],
            [42.197892278719166, 1.2463581562042239],
            [42.197892278719166, 1.2471413612365725],
        ], # ColldePaller
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ColldePaller\XMS-CAT_XMS-CAT_ColldePaller_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [41.913718967407085, 1.5264022350311282],
            [41.913718967407085, 1.527678966522217],
            [41.91400638646574, 1.5264022350311282],
            [41.91400638646574, 1.527678966522217],
        ], # ElMiracle
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\ElMiracle\XMS-CAT_XMS-CAT_ElMiracle_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.33592919742122, 3.0603575706481934],
            [42.33592919742122, 3.0639410018920903],
            [42.33748362505758, 3.0603575706481934],
            [42.33748362505758, 3.0639410018920903],
        ], # Garriguella
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Garriguella\XMS-CAT_XMS-CAT_Garriguella_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.14543614475607, 0.982019240182126],
            [42.14543614475607, 0.9836822097713595],
            [42.14651799247592, 0.982019240182126],
            [42.14651799247592, 0.9836822097713595],
        ], # LosCoscolls
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\LosCoscolls\XMS-CAT_XMS-CAT_LosCoscolls_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.217958388113736, 1.019185781478882],
            [42.217958388113736, 1.0197973251342776],
            [42.218458965975145, 1.019185781478882],
            [42.218458965975145, 1.0197973251342776],
        ], # Pessonada
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\Pessonada\XMS-CAT_XMS-CAT_Pessonada_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_sm_0.500000_0.500000_CS655_20171101_20241110.stm',
        'coordinates': [
            [42.210655856758265, 0.8932185173034669],
            [42.210655856758265, 0.8949351310729982],
            [42.21168891029004, 0.8932185173034669],
            [42.21168891029004, 0.8949351310729982],
        ], # SerradeCostaAmpla
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\XMS-CAT\SerradeCostaAmpla\XMS-CAT_XMS-CAT_SerradeCostaAmpla_static_variables.csv' 
    },

    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_sm_0.500000_0.500000_ThetaProbe-ML2X-B_20171101_20241110.stm',
        'coordinates': [
            [44.78995234463661, -0.57828426361084],
            [44.78995234463661, -0.5760955810546876],
            [44.79204620992998, -0.57828426361084],
            [44.79204620992998, -0.5760955810546876],
        ], # FR_Aqui
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\STEMS\CAN-CT-NT\STEMS_STEMS_CAN-CT-NT_sm_0.500000_0.500000_EC5_20171101_20241110.stm',
        'coordinates': [
            [44.681729523166624, 8.625973463058473],
            [44.681729523166624, 8.627743721008303],
            [44.68297294833046, 8.625973463058473],
            [44.68297294833046, 8.627743721008303],
        ], # CAN-CT-NT
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/STEMS/CAN-CT-NT/STEMS_STEMS_CAN-CT-NT_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Gevenich\TERENO_TERENO_Gevenich_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.98609893339354, 6.32340431213379],
            [50.98609893339354, 6.330077648162843],
            [50.988908469949415, 6.32340431213379],
            [50.988908469949415, 6.330077648162843],
        ], # Gevenich
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110/TERENO/Gevenich/TERENO_TERENO_Gevenich_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.92975076859899, 6.2960994243621835],
            [50.92975076859899, 6.297612190246583],
            [50.930312018942445, 6.2960994243621835],
            [50.930312018942445, 6.297612190246583],
        ], # Merzenhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Merzenhausen\TERENO_TERENO_Merzenhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_sm_0.500000_0.500000_Hydraprobe-II-Sdi-12-A_20171101_20241110.stm',
        'coordinates': [
            [50.868486478093025, 6.448953151702882],
            [50.868486478093025, 6.45124912261963],
            [50.869664616471276, 6.448953151702882],
            [50.869664616471276, 6.45124912261963],
        ], # Selhausen
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\TERENO\Selhausen\TERENO_TERENO_Selhausen_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_sm_0.450000_0.450000_TRIME-EZ_20171101_20241110.stm',
        'coordinates': [
            [52.165415885421794, 14.12341833834944],
            [52.165415885421794, 14.125242240479569],
            [52.16710714333233, 14.12341833834944],
            [52.16710714333233, 14.125242240479569],
        ], # Falkenberg
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\MOL-RAO\Falkenberg\MOL-RAO_MOL-RAO_Falkenberg_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [56.48257938387106, -3.1150746345520024],
            [56.48257938387106, -3.1124567985534672],
            [56.48428556158372, -3.1150746345520024],
            [56.48428556158372, -3.1124567985534672],
        ], # Balruddery
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Balruddery\COSMOS-UK_COSMOS-UK_Balruddery_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.86049592190234, -1.1268067359924319],
            [52.86049592190234, -1.1252081394195559],
            [52.86146755372759, -1.1268067359924319],
            [52.86146755372759, -1.1252081394195559],
        ], # BunnyPark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\BunnyPark\COSMOS-UK_COSMOS-UK_BunnyPark_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [52.105628764816664, -0.4271149635314942],
            [52.105628764816664, -0.4243040084838868],
            [52.106340450352924, -0.4271149635314942],
            [52.106340450352924, -0.4243040084838868],
        ], # Cardington
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\Cardington\COSMOS-UK_COSMOS-UK_Cardington_static_variables.csv' 
    },
    {
        'file_path': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_sm_0.000000_0.300000_Cosmic-ray-Probe_20171101_20241110.stm',
        'coordinates': [
            [55.215676089352854, -1.6985893249511719],
            [55.215676089352854, -1.6943085193634035],
            [55.21752449229229, -1.6985893249511719],
            [55.21752449229229, -1.6943085193634035],
        ], # CocklePark
        'static_variables': r'./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\COSMOS-UK\CocklePark\COSMOS-UK_COSMOS-UK_CocklePark_static_variables.csv' 
    },
    # Add more entries for all your files
]
# Function to fetch Sentinel-2 indices
def get_sentinel2_indices(polygon_coordinates, days_before_today):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    
    location_polygon = ee.Geometry.Polygon([polygon_coordinates])

    collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                  .filterDate(start_date_str, end_date_str)
                  .filterBounds(location_polygon)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
                  .map(lambda image: image.select()
                       .addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))
                       .addBands(image.normalizedDifference(['B8', 'B11']).rename('NDWI'))
                       .addBands(image.expression(
                           '((NIR + 0.1) - (SWIR2 + 0.02)) / ((NIR + 0.1) + (SWIR2 + 0.02))', 
                           {'NIR': image.select('B8'), 'SWIR2': image.select('B12')}
                       ).rename('GVMI'))))

    collection_data = collection.map(lambda image: ee.Feature(None, image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=location_polygon, scale=10, maxPixels=1e9)
        .set('date', image.date().format('YYYY-MM-dd'))))

    indices_data = ee.FeatureCollection(collection_data).getInfo()['features']
    
    # Safely create the dataframe only if expected columns are present
    df = pd.DataFrame([
        {**feature['properties'], 'date': pd.to_datetime(feature['properties']['date'])} 
        for feature in indices_data
    ])
    
    # Check if the expected columns are available; if not, skip this dataset
    required_columns = ['NDVI', 'NDWI', 'GVMI']
    if not all(col in df.columns for col in required_columns):
        print("Warning: Missing one or more Sentinel-2 indices (NDVI, NDWI, GVMI). Skipping this data.")
        return pd.DataFrame(columns=['date', 'NDVI', 'NDWI', 'GVMI'])  # Return an empty dataframe
    
    # Otherwise, process the data
    df = df[['date', 'NDVI', 'NDWI', 'GVMI']]
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    df.interpolate(method='time', inplace=True)
    df.dropna(inplace=True)
    
    return df.reset_index()

# Aggregate data function as before, no changes needed here for handling the missing columns

# Function to fetch weather data including forecasts
def get_weather_data(polygon_coordinates, days_before_today, forecast_days=3):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')
    
    lon = np.mean([point[0] for point in polygon_coordinates])
    lat = np.mean([point[1] for point in polygon_coordinates])
    parameters = "T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS"
    
    nasa_power_url_hist = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameters}"
        f"&community=AG&longitude={lon}&latitude={lat}"
        f"&start={start_date_str}&end={end_date_str}&format=JSON"
    )
    response_hist = requests.get(nasa_power_url_hist)
    weather_data_hist = response_hist.json()["properties"]["parameter"]
    
    dates_hist = list(weather_data_hist["T2M"].keys())
    weather_df_hist = pd.DataFrame({
        'date': pd.to_datetime(dates_hist, format='%Y%m%d'),
        'temperature': list(weather_data_hist["T2M"].values()),
        'precipitation': list(weather_data_hist["PRECTOTCORR"].values()),
        'irradiation': list(weather_data_hist["ALLSKY_SFC_SW_DWN"].values()),
        'humidity': list(weather_data_hist["RH2M"].values()),
        'wind_speed': list(weather_data_hist["WS10M"].values()),
        'pressure': list(weather_data_hist["PS"].values())
    })
    weather_df_hist.replace(-999.00, np.nan, inplace=True)
    weather_df_hist.set_index('date', inplace=True)
    weather_df_hist.sort_index(inplace=True)
    weather_df_hist.interpolate(method='time', inplace=True)
    weather_df_hist.dropna(inplace=True)
    weather_df_hist.reset_index(inplace=True)
    
    # Create shifted features for forecast days
    for i in range(1, forecast_days + 1):
        weather_df_hist[f'temperature_t+{i}'] = weather_df_hist['temperature'].shift(-i)
        weather_df_hist[f'precipitation_t+{i}'] = weather_df_hist['precipitation'].shift(-i)
        weather_df_hist[f'irradiation_t+{i}'] = weather_df_hist['irradiation'].shift(-i)
        weather_df_hist[f'humidity_t+{i}'] = weather_df_hist['humidity'].shift(-i)
        weather_df_hist[f'wind_speed_t+{i}'] = weather_df_hist['wind_speed'].shift(-i)
        weather_df_hist[f'pressure_t+{i}'] = weather_df_hist['pressure'].shift(-i)
    weather_df_hist = weather_df_hist.iloc[:-forecast_days]
    return weather_df_hist

# Example usage to aggregate all data
days_before_today = 255
forecast_days = 3

# Aggregate all data
full_data = aggregate_all_data(data_files_info, days_before_today, forecast_days)

# Display the first few rows
print(full_data.head())


Missing columns in ./Data_separate_files_header_20171101_20241110_11566_FOcc_20241110\FR_Aqui\parcmeteo\FR-Aqui_FR-Aqui_parcmeteo_static_variables.csv. Expected columns: 'quantity_name' and 'value'
             Date  Daily Average Soil Moisture     Location  clay fraction  \
0      2017-11-01                     0.087500  Villevielle          13.48   
1      2017-11-02                     0.088000  Villevielle          13.48   
2      2017-11-03                     0.087750  Villevielle          13.48   
3      2017-11-04                     0.088292  Villevielle          13.48   
4      2017-11-05                     0.237833  Villevielle          13.48   
...           ...                          ...          ...            ...   
63305  2022-12-27                     0.521000   CocklePark          25.00   
63306  2022-12-28                     0.531000   CocklePark          25.00   
63307  2022-12-29                     0.493000   CocklePark          25.00   
63308  2022-12-30     

In [ ]:
print(full_data)

         date      NDVI      NDWI      GVMI  Daily Average Soil Moisture  \
0  2024-03-07  0.019317 -0.139361 -0.062669                     0.466975   
1  2024-03-12  0.021721 -0.135051 -0.069282                     0.478913   
2  2024-03-17  0.040756 -0.122818 -0.070210                     0.474329   
3  2024-03-22  0.031723 -0.076312 -0.041156                     0.469663   
4  2024-03-27  0.017797 -0.148649 -0.068586                     0.471100   
5  2024-04-01  0.022165 -0.134367 -0.070273                     0.471571   
6  2024-04-06  0.020427 -0.142829 -0.069114                     0.462292   
7  2024-04-11  0.028820 -0.130976 -0.072791                     0.426325   
8  2024-04-21  0.012286 -0.147844 -0.066654                     0.441238   
9  2024-04-26  0.025180 -0.132316 -0.065941                     0.470133   
10 2024-05-01  0.020209 -0.140037 -0.072384                     0.451950   
11 2024-05-06  0.019034 -0.137941 -0.061888                     0.415171   
12 2024-05-1